# One-to-Many and Many-to-Many Joins - Lab

## Introduction

In this lab, you'll practice your knowledge on One-to-Many and Many-to-many relationships!

## Objectives

You will be able to:
- Query data including Many-to-Many relationships
- Write queries that make use of Join Tables

## One-to-Many and Many-to-Many Joins
<img src='Database-Schema.png' width=550>

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('data.sqlite', detect_types=sqlite3.PARSE_COLNAMES)
cur = conn.cursor()

## Employees and their Office (a One-to-One join)

Return a list of all of the employees with their first name, last name and the city and state of the office that they work out of (if they have one). Include all employees and order them by their first name, then their last name.

In [9]:
cur.execute('''SELECT employees.firstName, employees.lastName, offices.city, offices.state
                    FROM employees
                    JOIN offices
                    USING(officeCode)
                    ORDER BY employees.firstName, employees.lastName
            ;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print('Number of results:', len(df))
df.head()

Number of results: 23


,firstName,lastName,city,state
0,Andy,Fixter,Sydney,
1,Anthony,Bow,San Francisco,CA
2,Barry,Jones,London,
3,Diane,Murphy,San Francisco,CA
4,Foon Yue,Tseng,NYC,NY


## Customers and their Orders (a One-to-Many join)

Return a list of all of the customers. For each customer return a record for each of their order numbers, order dates and statuses.

In [10]:
cur.execute('''SELECT customers.customerName, orders.orderNumber, orders.orderDate, orders.status 
                FROM customers
                JOIN orders
                USING(customerNumber)
            ;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print('Number of results:', len(df))
df.head()

Number of results: 326


,customerName,orderNumber,orderDate,status
0,Atelier graphique,10123,2003-05-20,Shipped
1,Atelier graphique,10298,2004-09-27,Shipped
2,Atelier graphique,10345,2004-11-25,Shipped
3,Signal Gift Stores,10124,2003-05-21,Shipped
4,Signal Gift Stores,10278,2004-08-06,Shipped


## Orders and their Order Details (another One-to-Many join)

Return a list of orders. For each order return a record for each order detail within the order.

In [17]:
#Note that using * selects everything from orders and everything from orderdetails
cur.execute('''SELECT *
                FROM orders
                JOIN orderdetails
                USING(orderNumber)
            ;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print('Number of results:', len(df))
df.head()

Number of results: 2996


,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,S18_1749,30,136.00,3
1,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,S18_2248,50,55.09,2
2,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,S18_4409,22,75.46,4
3,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,S24_3969,49,35.29,1
4,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,S18_2325,25,108.06,4


In [16]:
#just to compare
cur.execute('''SELECT orderdetails.*
                FROM orderdetails
            ;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print('Number of results:', len(df))
df.head()

Number of results: 2996


,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2
2,10100,S18_4409,22,75.46,4
3,10100,S24_3969,49,35.29,1
4,10101,S18_2325,25,108.06,4


## Orders, Order details and Product Details (a Many-to-Many Join)

Return a list of the orders with the the order information, and allthe product information for each product in the order. 

In [18]:
cur.execute('''SELECT *
                FROM orders
                JOIN orderdetails
                USING(orderNumber)
                JOIN products
                USING(productCode)
            ;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print('Number of results:', len(df))
df.head()

Number of results: 2996


,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,S18_1749,30,136.00,3,1917 Grand Touring Sedan,Vintage Cars,1:18,Welly Diecast Productions,This 1:18 scale replica of the 1917 Grand Tour...,2724,86.70,170.00
1,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,S18_2248,50,55.09,2,1911 Ford Town Car,Vintage Cars,1:18,Motor City Art Classics,"Features opening hood, opening doors, opening ...",540,33.30,60.54
2,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,S18_4409,22,75.46,4,1932 Alfa Romeo 8C2300 Spider Sport,Vintage Cars,1:18,Exoto Designs,This 1:18 scale precision die cast replica fea...,6553,43.26,92.03
3,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,S24_3969,49,35.29,1,1936 Mercedes Benz 500k Roadster,Vintage Cars,1:24,Red Start Diecast,This model features grille-mounted chrome horn...,2081,21.75,41.03
4,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,S18_2325,25,108.06,4,1932 Model A Ford J-Coupe,Vintage Cars,1:18,Autoart Studio Design,This model features grille-mounted chrome horn...,9354,58.48,127.13


## Summary

In this lab, you practiced your knowledge on One-to-Many and Many-to-many relationships!